<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#F21DL-Data-Mining-&amp;-Machine-Learning" data-toc-modified-id="F21DL-Data-Mining-&amp;-Machine-Learning-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>F21DL Data Mining &amp; Machine Learning</a></span><ul class="toc-item"><li><span><a href="#Coursework-2:-Decision-Trees" data-toc-modified-id="Coursework-2:-Decision-Trees-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Coursework 2: Decision Trees</a></span><ul class="toc-item"><li><span><a href="#1.-Question-1." data-toc-modified-id="1.-Question-1.-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>1. Question 1.</a></span></li></ul></li><li><span><a href="#Question-3." data-toc-modified-id="Question-3.-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Question 3.</a></span></li></ul></li></ul></div>

# F21DL Data Mining & Machine Learning

## Coursework 2: Decision Trees
---
*ACCAD Dimitri, AUZIMOUR Antoine, DELTEL Clarence, DI MARTINO Thomas*

This notebook presents a python implementation for the Decision Trees classifier with the help of the input dataset.
### 1. Question 1.
<small><i>As we used Python to work on this Coursework, we answered to this question by loading and splitting the dataset using pandas and sklearn</i></small>


In [1]:
# MAIN IMPORTS

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import os
import numpy as np
from tqdm import tqdm
import math
import PIL

# SKLEARN IMPORTS

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder
import sklearn.preprocessing
import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier

# CONSTANTS
Y_TRAIN_ONE_VS_ALL_BASENAME =  "y_train_smpl_{}.csv"
SEED = 42
TEST_SIZE = 0.33
NUM_CLASSES = 10
DISCRETIZATION_BINS_NUMBER = 10
NUM_OF_FIELDS_TO_KEEP = 10
NUM_FIELDS = 10
NUM_FOLDS = 10
NUM_OF_TENS_OF_FIELDS_TO_TRAIN = 200
IMAGE_SHAPE = (48, 48)
CLASSES = {
    0: "speed_limit_60",
    1: "speed_limit_80",
    2: "speed_limit_80_lifted",
    3: "right of way at crossing",
    4: "right of way in general",
    5: "give way",
    6: "stop",
    7: "no speed limit general",
    8: "turn right down",
    9: "turn left down"
}

We now load our dataset from the .csv file: both the general discrimination and 1-vs-all problems.

In [2]:
data = pd.read_csv("x_train_gr_smpl.csv")
ground_truth = pd.read_csv("y_train_smpl.csv")
ground_truth.columns=["prediction"]
df = pd.concat([data, ground_truth], axis=1)

X_train, y_train = data, ground_truth

In [3]:
pd.read_csv("x_train_gr_smpl.csv").shape

(12660, 2304)

In [4]:
Y_ONE_VS_ALL = {}
# loading all one_vs_rest ground-truth
for i in CLASSES.keys():
    tmp_gt = pd.read_csv(Y_TRAIN_ONE_VS_ALL_BASENAME.format(i))
    # In 1 vs all, the "negative" instances are makred with 1, the psositive with 0
    # We change this to the opposite for convenience purposes 
    Y_ONE_VS_ALL[i] = (tmp_gt + 1 )%2 #transform 0s into 1s and 1s into 0s

## Question 3.
*Using the provided training data sets, and the 10-fold cross validation, run the classifier, and note its accuracy for varying learning parameters provided by Weka (or your other tool of choice). Record all your findings and explain them. Make sure you understand and can explainlogically the meaningof the confusion matrix, as well as the information containedin the “Detailed Accuracy” field: TP Rate, FP Rate, Precision, Recall, F Measure, ROC Area.*

In [8]:
tree_models = {
    "DecisionTree": {
        "base_class": tree.DecisionTreeClassifier,
        "models":[]
    }, # J48 is also known as C4.5 but scikit-learn only implements the CART algorithm, an evolution of the C4.5
    "ExtraTreesClassifier": {
        "base_class": tree.ExtraTreeClassifier,
        "models":[]
    },# similar to random forest in the way that it uses multiple decision trees with a voting system to determine the final class.
      # Their difference lies in the splits: when ExtraTrees try random splits, RandomForest is greedier and try all possible ones to find the best
    "RandomForest": {
        "base_class": RandomForestClassifier,
        "models":[]
    }

}

skf = StratifiedKFold(n_splits=NUM_FOLDS, shuffle=True, random_state=SEED)

# we fake the training data to give to the kfold: advised in the doc sklearn
fake_training_data = np.zeros(len(CLASSES.keys()))

for i, (train_idx, test_idx) in enumerate(skf.split(X_train, y_train)):

    n_img_train = len(train_idx)
    
    print("Running Fold", i+1, "/", NUM_FOLDS, "on ",n_img_train, " images")

    splitted_X_train = X_train.iloc[train_idx, :]
    splitted_Y_train = y_train.iloc[train_idx, :]

    splitted_X_test = X_train.iloc[test_idx, :]
    splitted_y_test = y_train.iloc[test_idx, :]
    
    # looping over every model, training them and storing their trained model in an array
    for model_key in tree_models.keys():
        print("\tTraining model ", model_key)

        # initiate an instance of the model class
        model = tree_models[model_key]["base_class"]()

        tree_models[model_key]["models"].append(model.fit(splitted_X_train, np.ravel(splitted_Y_train)))
        
        print("\tTesting model ", model_key)
        predicted_test = model.predict(splitted_X_test)
        
        # print test results
        print(metrics.classification_report(splitted_y_test, predicted_test))
    
    print('-----------------------')
    print('-----------------------')



Running Fold 1 / 10 on  11394  images
	Training model  DecisionTree
              precision    recall  f1-score   support

           0       0.80      0.78      0.79       141
           1       0.79      0.80      0.79       186
           2       0.80      0.93      0.86        42
           3       0.93      0.89      0.91       132
           4       0.93      0.95      0.94       210
           5       0.94      0.94      0.94       216
           6       0.96      0.90      0.93        78
           7       0.78      0.88      0.82        24
           8       0.96      0.94      0.95       207
           9       0.82      0.93      0.87        30

    accuracy                           0.89      1266
   macro avg       0.87      0.89      0.88      1266
weighted avg       0.89      0.89      0.89      1266

	Training model  ExtraTreesClassifier
              precision    recall  f1-score   support

           0       0.77      0.69      0.73       141
           1       0.75   

C:\Users\thoma\AppData\Local\Continuum\anaconda3\envs\python_gpu\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\thoma\AppData\Local\Continuum\anaconda3\envs\python_gpu\lib\site-packages\ipykernel_launcher.py:42: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


              precision    recall  f1-score   support

           0       0.92      0.94      0.93       141
           1       0.93      0.92      0.93       186
           2       1.00      0.93      0.96        42
           3       0.99      0.99      0.99       132
           4       0.98      1.00      0.99       210
           5       1.00      1.00      1.00       216
           6       1.00      1.00      1.00        78
           7       1.00      0.92      0.96        24
           8       1.00      1.00      1.00       207
           9       1.00      0.90      0.95        30

    accuracy                           0.97      1266
   macro avg       0.98      0.96      0.97      1266
weighted avg       0.97      0.97      0.97      1266

Running Fold 2 / 10 on  11394  images
	Training model  DecisionTree
              precision    recall  f1-score   support

           0       0.80      0.80      0.80       141
           1       0.80      0.82      0.81       186
          

C:\Users\thoma\AppData\Local\Continuum\anaconda3\envs\python_gpu\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\thoma\AppData\Local\Continuum\anaconda3\envs\python_gpu\lib\site-packages\ipykernel_launcher.py:42: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


              precision    recall  f1-score   support

           0       0.92      0.94      0.93       141
           1       0.93      0.96      0.94       186
           2       0.97      0.93      0.95        42
           3       0.98      0.98      0.98       132
           4       0.98      0.98      0.98       210
           5       0.99      0.99      0.99       216
           6       1.00      1.00      1.00        78
           7       1.00      1.00      1.00        24
           8       0.99      0.99      0.99       207
           9       1.00      0.90      0.95        30

    accuracy                           0.97      1266
   macro avg       0.98      0.96      0.97      1266
weighted avg       0.97      0.97      0.97      1266

Running Fold 3 / 10 on  11394  images
	Training model  DecisionTree
              precision    recall  f1-score   support

           0       0.75      0.80      0.77       141
           1       0.83      0.76      0.79       186
          

...

              precision    recall  f1-score   support

           0       0.89      0.94      0.91       141
           1       0.95      0.94      0.95       186
           2       0.93      0.88      0.90        42
           3       0.99      0.97      0.98       132
           4       0.98      0.98      0.98       210
           5       0.98      0.98      0.98       216
           6       0.99      0.99      0.99        78
           7       1.00      0.96      0.98        24
           8       0.99      0.98      0.98       207
           9       1.00      1.00      1.00        30

    accuracy                           0.97      1266
   macro avg       0.97      0.96      0.97      1266
weighted avg       0.97      0.97      0.97      1266

Running Fold 9 / 10 on  11394  images
	Training model  DecisionTree
              precision    recall  f1-score   support

           0       0.86      0.84      0.85       141
           1       0.83      0.83      0.83       186
          

C:\Users\thoma\AppData\Local\Continuum\anaconda3\envs\python_gpu\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\thoma\AppData\Local\Continuum\anaconda3\envs\python_gpu\lib\site-packages\ipykernel_launcher.py:42: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


              precision    recall  f1-score   support

           0       0.93      0.94      0.94       141
           1       0.91      0.95      0.93       186
           2       1.00      0.95      0.98        42
           3       0.98      0.98      0.98       132
           4       1.00      0.97      0.98       210
           5       1.00      0.99      0.99       216
           6       0.99      1.00      0.99        78
           7       0.96      0.96      0.96        24
           8       0.99      0.99      0.99       207
           9       1.00      1.00      1.00        30

    accuracy                           0.97      1266
   macro avg       0.98      0.97      0.97      1266
weighted avg       0.97      0.97      0.97      1266

Running Fold 10 / 10 on  11394  images
	Training model  DecisionTree
              precision    recall  f1-score   support

           0       0.85      0.78      0.81       141
           1       0.80      0.81      0.80       186
         

C:\Users\thoma\AppData\Local\Continuum\anaconda3\envs\python_gpu\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\thoma\AppData\Local\Continuum\anaconda3\envs\python_gpu\lib\site-packages\ipykernel_launcher.py:42: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


              precision    recall  f1-score   support

           0       0.93      0.95      0.94       141
           1       0.94      0.94      0.94       186
           2       1.00      0.98      0.99        42
           3       0.99      0.98      0.99       132
           4       0.98      1.00      0.99       210
           5       1.00      0.99      0.99       216
           6       1.00      0.96      0.98        78
           7       1.00      1.00      1.00        24
           8       1.00      1.00      1.00       207
           9       1.00      1.00      1.00        30

    accuracy                           0.98      1266
   macro avg       0.98      0.98      0.98      1266
weighted avg       0.98      0.98      0.98      1266

